# Sign Language Alphabet Detection - Training and TFLite Model Generation

This notebook **objectives** are:

1. Setup and configure Tensorflow Object Detection environment for training deep learning model.
2. Download and store Sign Language letters dataset.
3. Configure MobileNet SSDv2 model pipeline for training with such dataset,
4. Train model.
5. Export trained model to TFLite.

---

Code adapted from:
- https://github.com/luxonis/depthai-ml-training/tree/master/colab-notebooks.
- https://colab.research.google.com/github/luxonis/depthai-ml-training/blob/master/colab-notebooks/Easy_Object_Detection_Demo_Training.ipynb

Dataset from:
- https://public.roboflow.com/object-detection/american-sign-language-letters


## Install Training Environment Dependencies and Configs

In [ ]:
!python --version

In [ ]:
 ##!pip install --upgrade numpy
!pip install numpy==1.19.5

In [ ]:
# %%capture
#After this cell executes runtime will restart to finish the install, ignore and close the crash message, continue running cells starting with the one below
!pip install numpy==1.17.5;
import os
os.kill(os.getpid(), 9)

In [ ]:
%tensorflow_version 1.x
!pip install tf_slim

In [ ]:
# Number of training steps - 1000 will train very quickly, but more steps will increase accuracy.

num_steps = 60000  # A step means using a single batch of data. larger batch, less steps required
#60000 steps is required to train our example sign language dataset, less or more may be required based on your custom datasets needs

#Number of evaluation steps.
num_eval_steps = 50
#Batch size 24 is a setting that generally works well. can be changed higher or lower 
MODELS_CONFIG = {
        'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 24
    }
}
selected_model = 'ssd_mobilenet_v2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colab's GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Environment and Training Setup

Clone TF models which contains the Object Detection API; also install the required dependencies


### Only first time (no data stored in Drive)

In [ ]:
# %%capture
#Inside/ content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/
!git clone --quiet https://github.com/tensorflow/models.git
%cd "/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/models/"
!git checkout -f "58d19c67e1d30d905dd5c6e5092348658fed80af"

### Second time (/models) folder is present

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language"

In [ ]:
%cd models/

In [ ]:
!apt-get update && apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools

In [ ]:
%cd research

In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
import os
os.environ['PYTHONPATH'] += ':/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/models/research/://content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/models/research/slim/'
!python object_detection/builders/model_builder_test.py

In [ ]:
import os

repo_url = 'https://github.com/roboflow-ai/tensorflow-object-detection-faster-rcnn'

%cd "/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/"

repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))

In [ ]:
# Clone only first time, if tensorflow-object-detection-faster-rcnn folder not present
#!git clone {repo_url}
%cd {repo_dir_path}
#!git pull

### Prepare Sign Language `tfrecord` files (Dataset)



<img src='https://i5.walmartimages.com/asr/870a3fc2-f79a-4ff0-8d6d-c9af281ab883_2.2b4e10bcda389f068c94e38941b06500.jpeg' /> ```

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/tensorflow-object-detection-faster-rcnn/data"

In [ ]:
#Download sign language dataset from roboflow 
#   only if not already present tensorflow-object-detection-faster-rcnn/data

#!curl -L "https://public.roboflow.com/ds/oIXT8tYZhI?key=wh13ijZUHk" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
test_record_fname = "/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/tensorflow-object-detection-faster-rcnn/data/test/Letters.tfrecord"
train_record_fname = "/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/tensorflow-object-detection-faster-rcnn/data/train/Letters.tfrecord"
label_map_pbtxt_fname = "/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/tensorflow-object-detection-faster-rcnn/data/train/Letters_label_map.pbtxt"

We can take a look at the labels file

In [ ]:
%cat "/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/tensorflow-object-detection-faster-rcnn/data/train/Letters_label_map.pbtxt"

### Download the Mobilenet SSD v2 Model

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/models/research"

In [ ]:
import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = "/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/models/research/pretrained_model"

#### Only execute if not present

In [ ]:
if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)
!echo {DEST_DIR}

In [ ]:
!ls -la "/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/models/research/pretrained_model/"

### Configuring a Training Pipeline

The training pipeline configuration file indicates how the model training procedure will behave. Will setup train dataset location, number of classes, thresholds, etc.

In [ ]:
#TF pretrained model checkpoint
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

In [ ]:
pipeline_fname = os.path.join("/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/models/research/object_detection/samples/configs/", pipeline_file)

In [ ]:
import os

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [ ]:
import re
iou_threshold = 0.50
num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname) as f:
    print(pipeline_fname)
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    # Set number of classes num_classes.
    s = re.sub('iou_threshold: [0-9].[0-9]+',
               'iou_threshold: {}'.format(iou_threshold), s)
    
    f.write(s)

In [ ]:
pipeline_fname

In [ ]:
# #Have a look at the config file with various settings
!cat "/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config"

## Train Custom Mobilenet Model with Sign Language


In [ ]:
!pip uninstall pycocotools
!pip install --no-cache-dir pycocotools

In [ ]:
model_dir = 'training/'

In [ ]:
# Optionally remove content in output model directory for a fresh start.
#!rm -rf {model_dir}
#os.makedirs(model_dir, exist_ok=True)
!python "/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/models/research/object_detection/model_main.py" \
    --pipeline_config_path="/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config" \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps=60000 \
    --num_eval_steps={num_eval_steps}

#training will take a while (the TF OD library is not heavily optimized for speed on GPU), watch the mAP metrics rise, you can quit training early when you think your model has maxed out performance

W0325 15:09:23.563468 139746967033728 model_lib.py:717] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 60000
I0325 15:09:23.563827 139746967033728 config_util.py:552] Maybe overwriting train_steps: 60000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0325 15:09:23.564019 139746967033728 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0325 15:09:23.564181 139746967033728 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0325 15:09:23.564347 139746967033728 config_util.py:552] Maybe overwriting eval_num_epochs: 1
INFO:tensorflow:Maybe overwriting load_pretrained: True
I0325 15:09:23.564517 139746967033728 config_util.py:552] Maybe overwriting load_pretrained: True
INFO:tensorflow:Ignoring config override key: load_pretrained
I0325 15:09:23.564676 139746967033728 config_util.py

In [ ]:
#model dir check for the trained model
!ls {model_dir}

## Export Trained Model

### Obtain Trained Inference Graph
Once your training job is complete, you need to extract the newly trained inference graph, which will be later used to perform the object detection. This can be done as follows:

In [ ]:
#clean output_directory if necessary to start fresh:

# !rm -rf /content/object_detection_demo/fine_tuned_model/ 
# os.makedirs('/content/object_detection_demo_flow/fine_tuned_model/', exist_ok=True)

In [ ]:
import re
import numpy as np

#output_directory = './fine_tuned_model'
output_directory = '/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/exportedModel'

lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)

In [ ]:
!python '/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/models/research/object_detection/export_inference_graph.py' \
    --input_type=image_tensor \
    --pipeline_config_path='/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config' \
    --output_directory='/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/exportedModel' \
    --trained_checkpoint_prefix={last_model_path}

In [ ]:
!ls '/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/exportedModel'

In [ ]:
!ls '/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/exportedModel/saved_model'

### Export TFlite SSD Model Graph

In [ ]:
!python '/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/models/research/object_detection/export_tflite_ssd_graph.py' \
    --input_type=image_tensor \
    --pipeline_config_path='/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config' \
    --output_directory='/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/tflite_ssd_export_graph/' \
    --trained_checkpoint_prefix="training/model.ckpt-48400"

In [ ]:
!ls -l '/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/tflite_ssd_export_graph/'

In [ ]:
#import os
#pb_fname = os.path.join(os.path.abspath(output_directory), "saved_model/saved_model.pb")
#assert os.path.isfile(pb_fname), '`{}` not exist'.format(pb_fname)
# !ls -alh {pb_fname}

In [ ]:
#download frozen graph for posterity, you can keep this so you don't have to start over on training
#from google.colab import files
#files.download(pb_fname)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Convert Model Graph to TFLite Model File

For integrating the model in the Android application we will need to export the model to a Tensorflow lite model type

In [ ]:
!tflite_convert \
--graph_def_file '/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/tflite_ssd_export_graph/tflite_graph.pb' \
--output_file '/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/tflite_ssd_export_graph/model.tflite' \
--output_format "TFLITE" \
--input_arrays "normalized_input_image_tensor" \
--input_shapes "1,300,300,3" \
--inference_type "FLOAT" \
--output_arrays "TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3" \
--allow_custom_ops

In [ ]:
!ls '/content/drive/MyDrive/Colab Notebooks/Android TFLITE Sign/MobilenetSS2 Sign Language/tflite_ssd_export_graph/'